In [ ]:
# @title ## 3. Configure Docking Parameters 🎯 (Required)
# @markdown a. Please **click the Run button** ▶️ on the left side of the code cell and follow any prompts.
# @markdown
# @markdown b. After **uploading your receptor and ligand files** 📤, the system will automatically analyze and visualize them.
# @markdown
# @markdown c. The system will calculate optimal docking box parameters ⚙️ and generate 3D visualizations 🔍.

# Import necessary libraries
import os
import zipfile
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, FileLink
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from itertools import combinations, product
import uuid

if not os.path.exists('output'):
    os.makedirs('output')
    print("✅ Output directory created successfully!")

output_widget = widgets.Output()

upload_button_protein = widgets.FileUpload(
    description='Upload Receptor Files',
    accept='.pdb,.pdbqt,.zip',
    multiple=True
)

instructions_protein = """
### Upload Receptor Files 🏢
1. Upload single PDBQT/PDB file or ZIP containing multiple receptor files.
2. Files will be processed and visualized automatically.
"""

upload_button_ligand = widgets.FileUpload(
    description='Upload Ligand Files',
    accept='.pdb,.pdbqt',
    multiple=True  # Allow multiple file uploads
)

instructions_ligand = """
### Upload Ligand Files 💊
1. Upload one or more ligand PDBQT/PDB files.
2. Files will be processed and visualized automatically.
"""

display(Markdown(instructions_protein))
display(upload_button_protein)
display(Markdown(instructions_ligand))
display(upload_button_ligand)
display(output_widget)

def display_molecule(pdb_data, title="Molecular Structure", width=800, height=600):
    """Display molecular structure using 3Dmol.js"""

    container_id = f"3dmol_container_{str(uuid.uuid4()).replace('-', '_')}"

    html_content = f"""
    <div id="{container_id}" style="width: {width}px; height: {height}px; position: relative;">
        <p style="text-align: center; font-weight: bold; margin-bottom: 5px;">{title}</p>
    </div>
    <script src="https://3Dmol.org/build/3Dmol-min.js"></script>
    <script>
    $(document).ready(function() {{
        let viewer = $3Dmol.createViewer($("#{container_id}"), {{backgroundColor: 'white'}});
        let pdbData = `{pdb_data}`;

        viewer.addModel(pdbData, "pdb");
        viewer.setStyle({{}}, {{"cartoon": {{"color": "spectrum"}}}});
        viewer.zoomTo();
        viewer.render();
    }});
    </script>
    """

    return HTML(html_content)

def visualize_docking_3dmol(receptor_data, ligand_data, title="Docking Result", width=800, height=600):
    """Visualize docking results using 3Dmol.js"""

    container_id = f"docking_container_{str(uuid.uuid4()).replace('-', '_')}"

    html_content = f"""
    <div id="{container_id}" style="width: {width}px; height: {height}px; position: relative;">
        <p style="text-align: center; font-weight: bold; margin-bottom: 5px;">{title}</p>
    </div>
    <script src="https://3Dmol.org/build/3Dmol-min.js"></script>
    <script>
    $(document).ready(function() {{
        let viewer = $3Dmol.createViewer($("#{container_id}"), {{backgroundColor: 'white'}});

        // Add receptor
        let receptorData = `{receptor_data}`;
        let receptorModel = viewer.addModel(receptorData, "pdb");

        // Add ligand
        let ligandData = `{ligand_data}`;
        let ligandModel = viewer.addModel(ligandData, "pdb");

        // Set styles for receptor and ligand
        viewer.setStyle({{model: 0}}, {{"cartoon": {{color: "spectrum", opacity: 0.8}}}});
        viewer.setStyle({{model: 1}}, {{"stick": {{colorscheme: "greenCarbon", radius: 0.2}}}});

        // Add surface
        viewer.addSurface($3Dmol.SurfaceType.MS, {{"opacity": 0.3, "color": "white"}}, {{model: 0}});

        viewer.zoomTo();
        viewer.render();
    }});
    </script>
    """

    return HTML(html_content)

def parse_pdb(file_path):
    atoms = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('ATOM') or line.startswith('HETATM'):
                try:
                    atom_info = {
                        'atom_name': line[12:16].strip(),
                        'residue_name': line[17:20].strip(),
                        'chain_id': line[21].strip(),
                        'residue_seq': int(line[22:26].strip()),
                        'x': float(line[30:38].strip()),
                        'y': float(line[38:46].strip()),
                        'z': float(line[46:54].strip())
                    }
                    atoms.append(atom_info)
                except (ValueError, IndexError):
                    continue
    return atoms

extracted_files = []

def process_protein_files(change):
    output_widget.clear_output()  
    if not upload_button_protein.value:
        output_widget.append_stdout("⚠️ Please upload receptor files\n")
        return  

    global extracted_files
    extracted_files = []
    for file_name, file_info in upload_button_protein.value.items():
        with open(file_name, 'wb') as f:
            f.write(file_info['content'])
        output_widget.append_stdout(f"✅ Receptor uploaded: {file_name}\n")

        if file_name.endswith('.zip'):
            output_widget.append_stdout(f"🔄 Extracting PDB/PDBQT files from {file_name}...\n")
            with zipfile.ZipFile(file_name, 'r') as zip_ref:
                zip_ref.extractall()
                extracted_files.extend([name for name in zip_ref.namelist() if name.endswith(('.pdb', '.pdbqt'))])
        else:
            extracted_files.append(file_name)

    for pdb_file in extracted_files:
        try:
            with open(pdb_file, 'r') as f:
                pdb_data = f.read()
            output_widget.append_stdout(f"🔍 Visualizing: {pdb_file}\n")
            display(display_molecule(pdb_data, title=f"Receptor: {os.path.basename(pdb_file)}"))
        except Exception as e:
            output_widget.append_stdout(f"❌ Error visualizing file {pdb_file}: {str(e)}\n")

upload_button_protein.observe(process_protein_files, names='value')

def process_ligand_files(change):
    if not upload_button_ligand.value:
        output_widget.append_stdout("⚠️ Please upload ligand files\n")
        return  

    ligand_files = []
    for file_name, file_info in upload_button_ligand.value.items():
        with open(file_name, 'wb') as f:
            f.write(file_info['content'])
        output_widget.append_stdout(f"✅ Ligand uploaded: {file_name}\n")
        ligand_files.append(file_name)

        try:
            with open(file_name, 'r') as f:
                pdb_data = f.read()
            output_widget.append_stdout(f"🔍 Visualizing: {file_name}\n")
            display(display_molecule(pdb_data, title=f"Ligand: {os.path.basename(file_name)}", width=600, height=400))
        except Exception as e:
            output_widget.append_stdout(f"❌ Error visualizing file {file_name}: {str(e)}\n")

    receptors = [file for file in extracted_files if file.endswith(('.pdb', '.pdbqt'))]
    ligands = ligand_files

    if receptors and ligands:
        output_params_and_images(receptors, ligands)
        output_widget.append_stdout("✅ Receptor, ligand and docking box visualization complete.\n")
    else:
        output_widget.append_stdout("⚠️ Please ensure both receptor and ligand files are uploaded.\n")

upload_button_ligand.observe(process_ligand_files, names='value')

def visualize_docking_single_box(receptor_file, ligand_file, output_dir):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    receptor_atoms = parse_pdb(receptor_file)
    x_receptor = [atom['x'] for atom in receptor_atoms]
    y_receptor = [atom['y'] for atom in receptor_atoms]
    z_receptor = [atom['z'] for atom in receptor_atoms]

    ligand_atoms = parse_pdb(ligand_file)
    x_ligand = [atom['x'] for atom in ligand_atoms]
    y_ligand = [atom['y'] for atom in ligand_atoms]
    z_ligand = [atom['z'] for atom in ligand_atoms]

    all_x = x_receptor + x_ligand
    all_y = y_receptor + y_ligand
    all_z = z_receptor + z_ligand

    if not all_x or not all_y or not all_z:
        raise ValueError("Unable to extract atom coordinates from files")

    center_x = np.mean(all_x)
    center_y = np.mean(all_y)
    center_z = np.mean(all_z)
    size_x = max(all_x) - min(all_x) + 10  
    size_y = max(all_y) - min(all_y) + 10
    size_z = max(all_z) - min(all_z) + 10

    box_min_x, box_max_x = center_x - size_x / 2, center_x + size_x / 2
    box_min_y, box_max_y = center_y - size_y / 2, center_y + size_y / 2
    box_min_z, box_max_z = center_z - size_z / 2, center_z + size_z / 2

    r = [box_min_x, box_max_x]
    for s, e in combinations(np.array(list(product(r, r, r))), 2):
        if np.sum(np.abs(s-e)) == r[1]-r[0]:
            ax.plot3D(*zip(s, e), color="#AC99D2", alpha=0.6)  # Border color and opacity

    xx, yy = np.meshgrid([box_min_x, box_max_x], [box_min_y, box_max_y])
    zz = np.array([[box_min_z, box_min_z], [box_min_z, box_min_z]])
    ax.plot_surface(xx, yy, zz, color="#AC99D2", alpha=0.1)
    zz = np.array([[box_max_z, box_max_z], [box_max_z, box_max_z]])
    ax.plot_surface(xx, yy, zz, color="#AC99D2", alpha=0.1)
    yy, zz = np.meshgrid([box_min_y, box_max_y], [box_min_z, box_max_z])
    xx = np.array([[box_min_x, box_min_x], [box_min_x, box_min_x]])
    ax.plot_surface(xx, yy, zz, color="#AC99D2", alpha=0.1)
    xx = np.array([[box_max_x, box_max_x], [box_max_x, box_max_x]])
    ax.plot_surface(xx, yy, zz, color="#AC99D2", alpha=0.1)
    xx, zz = np.meshgrid([box_min_x, box_max_x], [box_min_z, box_max_z])
    yy = np.array([[box_min_y, box_min_y], [box_min_y, box_min_y]])
    ax.plot_surface(xx, yy, zz, color="#AC99D2", alpha=0.1)
    yy = np.array([[box_max_y, box_max_y], [box_max_y, box_max_y]])
    ax.plot_surface(xx, yy, zz, color="#AC99D2", alpha=0.1)

    ax.scatter(x_receptor, y_receptor, z_receptor,
               label=os.path.basename(receptor_file).replace('.pdb', '').replace('.pdbqt', ''),
               color='#8FB4DC', alpha=0.2)

    ax.scatter(x_ligand, y_ligand, z_ligand,
               label=os.path.basename(ligand_file).replace('.pdb', '').replace('.pdbqt', ''),
               color='#70CDBE', alpha=1.0, s=40, marker='^')  # Different color and style

    ax.set_xlim([box_min_x-10, box_max_x+10])
    ax.set_ylim([box_min_y-10, box_max_y+10])
    ax.set_zlim([box_min_z-10, box_max_z+10])

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'{os.path.basename(receptor_file).replace(".pdb", "").replace(".pdbqt", "")}_{os.path.basename(ligand_file).replace(".pdb", "").replace(".pdbqt", "")}')
    ax.legend()

    ax.grid(True)
    ax.view_init(elev=20., azim=-35)

    img_filename = os.path.join(output_dir, f'{os.path.basename(receptor_file).replace(".pdb", "").replace(".pdbqt", "")}_{os.path.basename(ligand_file).replace(".pdb", "").replace(".pdbqt", "")}.png')
    plt.savefig(img_filename, dpi=300)
    plt.close()

    try:
        with open(receptor_file, 'r') as f:
            receptor_data = f.read()
        with open(ligand_file, 'r') as f:
            ligand_data = f.read()

        display(visualize_docking_3dmol(
            receptor_data,
            ligand_data,
            title=f"Docking: {os.path.basename(receptor_file)} + {os.path.basename(ligand_file)}"
        ))
    except Exception as e:
        output_widget.append_stdout(f"❌ Error visualizing docking with 3Dmol.js: {str(e)}\n")

    return img_filename, center_x, center_y, center_z, size_x, size_y, size_z

def output_params_and_images(receptors, ligands):
    if not os.path.exists('output'):
        os.makedirs('output')

    params_files = []
    img_files = []

    for receptor in receptors:
        for ligand in ligands:
            try:
                output_widget.append_stdout(f"🔄 Processing: {os.path.basename(receptor)} + {os.path.basename(ligand)}\n")

                img_filename, center_x, center_y, center_z, size_x, size_y, size_z = visualize_docking_single_box(
                    receptor, ligand, 'output'
                )

                params_filename = os.path.join('output', f'{os.path.basename(receptor).replace(".pdb", "").replace(".pdbqt", "")}_{os.path.basename(ligand).replace(".pdb", "").replace(".pdbqt", "")}.txt')
                with open(params_filename, 'w') as f:
                    f.write(f'Receptor: {receptor}\n')
                    f.write(f'Ligand: {ligand}\n')
                    f.write('Docking Type: Blind Docking\n')
                    f.write(f'center_x = {center_x:.3f}\n')
                    f.write(f'center_y = {center_y:.3f}\n')
                    f.write(f'center_z = {center_z:.3f}\n')
                    f.write(f'size_x = {size_x:.3f}\n')
                    f.write(f'size_y = {size_y:.3f}\n')
                    f.write(f'size_z = {size_z:.3f}\n')

                params_files.append(params_filename)
                img_files.append(img_filename)

                output_widget.append_stdout(f"📊 Docking Parameters ({os.path.basename(receptor)} + {os.path.basename(ligand)}):\n")
                output_widget.append_stdout(f"  Center coordinates: ({center_x:.3f}, {center_y:.3f}, {center_z:.3f})\n")
                output_widget.append_stdout(f"  Size: ({size_x:.3f}, {size_y:.3f}, {size_z:.3f})\n")

            except Exception as e:
                output_widget.append_stdout(f"❌ Error processing {receptor} and {ligand}: {str(e)}\n")

    if params_files and img_files:
        with zipfile.ZipFile('Docking_parameter.zip', 'w') as zipf:
            for file in params_files + img_files:
                zipf.write(file)

        output_widget.append_stdout("📦 Output files generated:\n")
        display(FileLink('Docking_parameter.zip'))

        files.download('Docking_parameter.zip')
    else:
        output_widget.append_stdout("⚠️ No parameter or image files were generated. Please check your input files.\n")

display(output_widget)